# Predicting Load demand using Weather 

We wish to see the correlation between weather and electricity load demand in the NY state. The weather and load data were obtained from ~link~ ~link~

This data was processed and the combined to get a consolidated data source which we will use in this notebook to predict the load demand using Random Forest.

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import xgboost 
import dask_cudf
from dask import delayed
import dask_xgboost
from dask.distributed import Client, wait
from dask.dataframe import from_delayed
import cudf
import dask
from dask_cuda import LocalCUDACluster
import numpy as np
import pandas as pd


cluster = LocalCUDACluster()
client = Client(cluster)

# Reading Data

Let's read in the data and take a look at the available fields

In [3]:
merged_ddf = dask_cudf.read_csv("processed_data.csv")
merged_ddf.head().to_pandas()

,Unnamed: 0,county,day,month,hour,year,Load_x,air_tmp_x,dew_x,sea_pressure_x,...,wind_spd_tmp,season,Weekday,Load_x_t-1,air_tmpt-1,wind_spd_t-1,Load_x_5,air_tmp_x_5,sea_pressure_x_5,wind_spd_x_5
0,2595,0.0,20.0,9.0,12.0,2019.0,4009.800049,233.0,-14.0,10137.0,...,0.064377682,2.0,1.0,-1.000000,-1.0,-1.0,0.000000,0.0,0.0,0.0
1,2594,0.0,20.0,9.0,11.0,2019.0,9195.900391,217.0,-13.0,10140.0,...,0.096774194,2.0,1.0,4009.800049,233.0,15.0,0.000000,0.0,0.0,0.0
2,2593,0.0,20.0,9.0,10.0,2019.0,9097.000000,217.0,-13.0,10139.0,...,0.069124424,2.0,1.0,9195.900391,217.0,21.0,0.000000,0.0,0.0,0.0
3,2592,0.0,20.0,9.0,9.0,2019.0,7576.100098,67.0,-999.0,10178.0,...,0.611940299,2.0,1.0,9097.000000,217.0,15.0,0.000000,0.0,0.0,0.0
4,2591,0.0,20.0,9.0,8.0,2019.0,7330.799805,67.0,-999.0,10178.0,...,0.537313433,2.0,1.0,7576.100098,67.0,41.0,7441.920069,160.2,10154.4,25.6


## Brief about some fields

The county field is a categorical variable that represents the county in NY state that the entry belongs to. 

`day`, `month`, `year`, `hour` represent the `TimeStamp`(This column was split to obtain the other fields) the record was taken.

`Load_x` is the actual value for Load consumed at the time. Similarly `air_tmp_x`, `dew_x` ... are the values recorded at the given time.

`Load_x_t-1` is the load recorded at `t-1` time. It was calculated per county to maintain correctness. (Same is the case for all columns ended with `t-1` 

`Load_x_5` is the summary of theprevious 5 entries (calculated using `rolling`) 

### Uninterested columns

We will exclude `Unnamed: 0`, `precip_6_x` (because most of the values are null) and `TimeStamp` because we already have the fields representing this value making this redundant.

In [4]:
from dask_ml.model_selection import train_test_split

def split_data(merged_ddf):
    X_train = merged_ddf[input_cols].loc[:int(0.8*len(merged_ddf))]
    y_train = merged_ddf['Load_x'].loc[:int(0.8*len(merged_ddf))]
    
    X_test = merged_ddf[input_cols].loc[int(0.8*len(merged_ddf)):]
    y_test= merged_ddf['Load_x'].loc[int(0.8*len(merged_ddf)):]
    
    print("Train len ", len(X_train))
    print("Test len ", len(X_test))
    
    train_dmat = xgboost.DMatrix(X_train.compute(), y_train.compute())
    test_dmat = xgboost.DMatrix(X_test.compute(), y_test.compute())
    return train_dmat, test_dmat, X_train, X_test, y_train, y_test

In [5]:
input_cols = [c for c in merged_ddf.columns if c not in['precip_6_x', 'TimeStamp', 'Load_x', 'Unnamed: 0']]

for col in input_cols:
    merged_ddf[col] = merged_ddf[col].astype('float32')

train_dmat, test_dmat, X_train, X_test, y_train, y_test = split_data(merged_ddf)

Train len  41222
Test len  10306


# Random Forest

In order to implement RF using xgboost, the following need to be set :

`booster` should be set to `gbtree`, as we are training forests. Note that as this is the default, this parameter needn’t be set explicitly.

`subsample` must be set to a value less than 1 to enable random selection of training cases (rows).

One of `colsample_by*` parameters must be set to a value less than 1 to enable random selection of columns. Normally, colsample_bynode would be set to a value less than 1 to randomly sample columns at each tree split.

`num_parallel_tree` should be set to the size of the forest being trained.

`num_boost_round` should be set to 1 to prevent XGBoost from boosting multiple random forests. Note that this is a keyword argument to train(), and is not part of the parameter dictionary.

eta (alias: `learning_rate`) must be set to 1 when training random forest regression.

random_state can be used to seed the random number generator.

In [6]:
rf_gpu_parameters = {'colsample_bynode': 0.9,
 'learning_rate': 1,
 'max_depth': 7,
 'num_parallel_tree': 100,
 'objective': 'reg:squarederror',
 'subsample': 0.6,
 'tree_method': 'gpu_hist',
 'min_child_weight': 6,
 'gamma': 0.1,
 'alpha': 1.0,
 'reg_lambda': 1.0,
}


In [7]:
rf_model = xgboost.train(
    rf_gpu_parameters,
    train_dmat,
    num_boost_round=1,
    evals=[(test_dmat, "Test"), (train_dmat, "Train")],
    early_stopping_rounds=10,
    verbose_eval=10
)

[0]	Test-rmse:1394.13599	Train-rmse:2864.63769
Multiple eval metrics have been passed: 'Train-rmse' will be used for early stopping.

Will train until Train-rmse hasn't improved in 10 rounds.


## Tuning Max Depth and Min Child Weight

In [8]:
def grid_search_depth_wt(gridsearch_params, train_dmat, xgb_gpu_params):

    min_rmse = float("Inf")
    best_params = None
    for i, (max_depth, min_child_weight) in enumerate(gridsearch_params):
        print("CV with max_depth={}, min_child_weight={}".format(
                                 max_depth,
                                 min_child_weight))
        # Update our parameters
        rf_gpu_parameters['max_depth'] = max_depth
        rf_gpu_parameters['min_child_weight'] = min_child_weight

        # Run CV
        cv_results = xgboost.cv(
            rf_gpu_parameters,
            train_dmat,
            num_boost_round=1,
            seed=42,
            nfold=3,
            metrics={'rmse'},
            early_stopping_rounds=10
        )
        # Update best MAE
        mean_rmse = cv_results['test-rmse-mean'].min()
        boost_rounds = cv_results['test-rmse-mean'].argmin()

        print("\tRMSE {} for {} rounds".format(mean_rmse, boost_rounds))
        if mean_rmse < min_rmse:
            min_rmse = mean_rmse
            best_params = (max_depth,min_child_weight)
    print("Best params: {}, {}, RMSE: {}".format(best_params[0], best_params[1],  min_rmse))
    return best_params

In [9]:
gridsearch_params = [(depth, child_wt) 
                     for depth in range(5,8) 
                     for child_wt in range(4,7)
                    ]

best_params = grid_search_depth_wt(gridsearch_params, train_dmat, rf_gpu_parameters)

CV with max_depth=5, min_child_weight=4
	RMSE 3059.5302733333338 for 0 rounds
CV with max_depth=5, min_child_weight=5
	RMSE 3059.7128903333337 for 0 rounds
CV with max_depth=5, min_child_weight=6
	RMSE 3060.157063666667 for 0 rounds
CV with max_depth=6, min_child_weight=4
	RMSE 3009.5072426666666 for 0 rounds
CV with max_depth=6, min_child_weight=5
	RMSE 3008.836914333333 for 0 rounds
CV with max_depth=6, min_child_weight=6
	RMSE 3008.546387 for 0 rounds
CV with max_depth=7, min_child_weight=4
	RMSE 2988.053711 for 0 rounds
CV with max_depth=7, min_child_weight=5
	RMSE 2997.6018063333336 for 0 rounds
CV with max_depth=7, min_child_weight=6
	RMSE 2989.198079666666 for 0 rounds
Best params: 7, 4, RMSE: 2988.053711


In [10]:
rf_gpu_parameters['max_depth'], rf_gpu_parameters['min_child_weight']= best_params[0], best_params[1]

In [11]:
min_rmse = float("Inf")
best_params = None
alpha_lambda = [(alpha, lam) 
               for alpha in [ 2.0, 5.0, 10.]
               for lam in [0.5, 1.5, 2.5]]
for alpha, lam in alpha_lambda:
    print("CV with alpha={}, lambda={}".format(alpha,lam))
    # Update our parameters
    rf_gpu_parameters['alpha'] = alpha
    rf_gpu_parameters['reg_lambda'] =lam
    # Run CV
    cv_results = xgboost.cv(
        rf_gpu_parameters,
        train_dmat,
        num_boost_round=1,
        seed=42,
        nfold=3,
        metrics={'rmse'},
        early_stopping_rounds=10
    )
    # Update best MAE
    mean_rmse = cv_results['test-rmse-mean'].min()
    boost_rounds = cv_results['test-rmse-mean'].argmin()

    
    print("\tRMSE {} for {} rounds".format(mean_rmse, boost_rounds))
    if mean_rmse < min_rmse:
        min_rmse = mean_rmse
        best_params = (alpha,lam)
print("Best params: {}, {} RMSE: {}".format(best_params[0], best_params[1] , min_rmse))

CV with alpha=2.0, lambda=0.5
	RMSE 2818.268229 for 0 rounds
CV with alpha=2.0, lambda=1.5
	RMSE 3106.5253909999997 for 0 rounds
CV with alpha=2.0, lambda=2.5
	RMSE 3247.151937 for 0 rounds
CV with alpha=5.0, lambda=0.5
	RMSE 2818.1606446666665 for 0 rounds
CV with alpha=5.0, lambda=1.5
	RMSE 3106.5264483333335 for 0 rounds
CV with alpha=5.0, lambda=2.5
	RMSE 3247.1400553333333 for 0 rounds
CV with alpha=10.0, lambda=0.5
	RMSE 2818.1315916666667 for 0 rounds
CV with alpha=10.0, lambda=1.5
	RMSE 3106.5394693333333 for 0 rounds
CV with alpha=10.0, lambda=2.5
	RMSE 3247.2327473333335 for 0 rounds
Best params: 10.0, 0.5 RMSE: 2818.1315916666667


In [12]:
rf_gpu_parameters['alpha'], rf_gpu_parameters['reg_lambda'] = best_params[0], best_params[1]

In [13]:
model = xgboost.train(
    rf_gpu_parameters                                                         ,
    train_dmat,
    num_boost_round=1,
    evals=[(test_dmat, "Test"), (train_dmat, "Train")],
    verbose_eval= 10,
    early_stopping_rounds=10
)

[0]	Test-rmse:1366.50000	Train-rmse:2671.21069
Multiple eval metrics have been passed: 'Train-rmse' will be used for early stopping.

Will train until Train-rmse hasn't improved in 10 rounds.


In [14]:
rf_gpu_parameters['num_parallel_tree'] = 1000
rf_gpu_parameters

{'colsample_bynode': 0.9,
 'learning_rate': 1,
 'max_depth': 7,
 'num_parallel_tree': 1000,
 'objective': 'reg:squarederror',
 'subsample': 0.6,
 'tree_method': 'gpu_hist',
 'min_child_weight': 4,
 'gamma': 0.1,
 'alpha': 10.0,
 'reg_lambda': 0.5}

In [15]:
model = xgboost.train(
    rf_gpu_parameters                                                         ,
    train_dmat,
    num_boost_round=1,
    evals=[(test_dmat, "Test"), (train_dmat, "Train")],
    verbose_eval= 10,
    early_stopping_rounds=10
)

[0]	Test-rmse:1364.63977	Train-rmse:2665.48877
Multiple eval metrics have been passed: 'Train-rmse' will be used for early stopping.

Will train until Train-rmse hasn't improved in 10 rounds.
